# Pabbi Land Cover Classification 

## Introduction:
This is a short analysis of the 5 Mauzas Cluster of Tehsil Pabbi, District Nowshera, Khybre Pakhtunkhwa Pakistan.The vector dataset of these mauzas (Revenue Estates) is provided by Mr. Aftab Ahmad Deputy Director GIS, BOR, KP for this analysis. The dataset consist of agriculture field boundries (parcels or Khasra in local revenue language). The major categories of the parcels are `Agriculture` , `Built Up`, `Stream`, `Roads/Streets`, `Graveyard` and `Others`. It is worth mentioning that the dataset is created based upon the manual settlement record of 1926 and the current ground situation might be different. This study highlights the agriculture parcles with a median NDVI value between 0.3 and 0.7 for the peroid `01-01-2024` to `31-12-2024`. 

## Datasets:
- Vector Dataset : Tis dataset is provided by [Mr. Aftab Ahamd](https://www.linkedin.com/in/aftab-ahmad-4316a463/), Deputy Director GIS, BOR, KP. The details are as follow:

| Land Use      | Number    | Area (Kanal)|
| --------      | -------   | --------    |
| Agriculture   | 5795      |32729.0      |
| Stream        | 302       |             |
| Other         | 229       |             |
|Road/Street    | 172       |             |
| Graveyard     | 29        |             |
| Built Up      | 24        |             |

- Satellite Imagery: [Sentinel-2 Multispectrum Harmonized](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED) imagery has been used for this analysis.

## Objectives:
1. To visualize the Cadastral(vector dataset) of 5 Mauzas on an interactive map
2. To visualize the major land use on seperate layers.
3. To visualize the median NDVI  of the area for `1 Year` from `01-01-2024` to `31-12-2024`using Harmonized Sentinel-2 (10 m satellite imagery). 
4. Plotting parcels which have ndvi value between .3 and 0.7 for the peroid


In [1]:
import ee
import geemap
import geopandas as gpd

In [2]:
geemap.ee.Initialize()

In [3]:
#parquet_url = 'https://github.com/code4geoai/gee/releases/download/0.2/pabbi.parquet'
parquet_file = 'pabbi.parquet'
# The parquet file must be local or can be read from url with request. lets use the local file.


In [4]:
gdf = gpd.read_parquet(parquet_file)
gdf['Area_m2'] = gdf['geometry'].area
gdf.head()

,Mouza_Name,Landuse_Ma,Area_Acre,FFID,Parcel_ID,geometry,Area_m2
0,Khushmaqam,Agriculture,0.101785,1,668.0,"MULTIPOLYGON (((753923.077 3769111.141, 753894...",411.907595
1,Khushmaqam,Built up,0.036718,2,670.0,"MULTIPOLYGON (((753959.894 3769126.291, 753959...",148.590279
2,Khushmaqam,Agriculture,0.315557,3,632.0,"MULTIPOLYGON (((753839.267 3769129.693, 753841...",1277.014013
3,Khushmaqam,Agriculture,0.187644,4,669.0,"MULTIPOLYGON (((753952.852 3769131.737, 753946...",759.369193
4,Khushmaqam,Agriculture,0.161568,5,693.0,"MULTIPOLYGON (((754361.202 3769166.424, 754269...",653.842075


In [6]:
# Group the area by Landuse_Ma value
gdf_grouped = gdf.groupby('Landuse_Ma')['Area_m2'].sum().reset_index()
# Now convert square meters to Kanals (1 Kanal = 505.857 m²)
gdf_grouped['Area_Kanal'] = gdf_grouped['Area_m2'] / 505.857
gdf_grouped.round()


,Landuse_Ma,Area_m2,Area_Kanal
0,Agriculture,16556433.0,32729.0
1,Built up,304364.0,602.0
2,Graveyard,101968.0,202.0
3,Other,529528.0,1047.0
4,Road/Streets,843873.0,1668.0
5,Stream,415791.0,822.0


In [5]:

# Convert the GeoDataFrame to an Earth Engine FeatureCollection
pabbi= geemap.gdf_to_ee(gdf)


In [7]:

#Filtering the features having Landuse_Ma as Builtup
builtup=gdf[gdf['Landuse_Ma'] == 'Built up']

builtstyle = {'color': 'white', 
         'width': 1,
         'fillColor': '00000000',
            
         }

In [8]:
#Filtering the features having Landuse_Ma as Agriculture
agri=gdf[gdf['Landuse_Ma'] == 'Agriculture']
agri_feature = geemap.gdf_to_ee(agri)
agristyle = {'color': 'yellow', 
         'width': 1,
         'fillColor': '00000000',
            
         }

In [9]:
#Filtering the features having Landuse_Ma as Graveyard
graveyard=gdf[gdf['Landuse_Ma'] == 'Graveyard']

gravestyle = {'color': 'black', 
         'width': 1,
         'fillColor': '00000000',
            
         }

In [10]:
# Basic Style for Pabbi Vector feature
style = {'color': 'blue', 
         'width': 1,
         'fillColor': '00000000',
             
         }

In [11]:
p = geemap.Map(fullscreen=True)
p.add_basemap('HYBRID')

In [12]:
# Adding Satellite Imagery and calculating NDVI using Sentinel-2
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
# Filter the collection for a specific region and time period
start_date = '2024-01-01'
end_date = '2024-12-31'
# Filter the collection by date and region
filtered_collection = collection.filterDate(start_date, end_date).filterBounds(pabbi.geometry()).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Select the first image from the filtered collection
median_image = filtered_collection.median().clip(pabbi.geometry())

# Calculate NDVI
ndvi = median_image.normalizedDifference(['B8', 'B4']).rename('NDVI')


# Select the bands of interest (e.g.,NVDI)
ndvi_vis = {
    'min': 0.25,
    'max': 0.8,
    'palette': ['white', 'yellow', 'green', 'red']  # Red for highest vegetation
}



# Selecting only those parcels which have ndvi value between .3 and 0.7 

In [13]:
# Step 1: Calculate NDVI for entire area alrady done above

# Step 2: Calculate mean NDVI for each agri polygon
agri_with_ndvi = ndvi.reduceRegions(
    collection=agri_feature,
    reducer=ee.Reducer.mean(),
    scale=10,
)

# Step 3: Filter polygons with NDVI between 0.3 and 0.7
agri_ndvi_filtered = agri_with_ndvi.filter(
    ee.Filter.And(
        ee.Filter.gte('mean', 0.3),
        ee.Filter.lte('mean', 0.7)
    )
)

In [16]:
# Style the filtered pabbi
agrindvi_style = {'color': 'red', 'fillColor': 'FFFF0040'}

In [ ]:
# Adding Layers to Map

p.addLayer(ndvi, ndvi_vis,  name='Median NDVI') # Add NDVI layer
#p.addLayer(pabbi.style(**style), {}, 'Pabbi Mouzas') # Add Pabbi layer Vector 
#p.add_gdf(builtup, layer_name='Built up', style=builtstyle) # Add Builtup layer

p.add_layer(agri_feature.style(**agristyle),{}, name='Agriculture') # Add Agriculture layer

#p.add_gdf(graveyard, layer_name='Graveyard', style=gravestyle) # Add Graveyard layer

p.addLayer(agri_ndvi_filtered.style(**agrindvi_style), {}, 'Agri-NDVI (0.3-0.7)Filtered')

p.add_colorbar(
    vis_params=ndvi_vis,
    label='NDVI',
    orientation='horizontal',
    position='bottomright',
    layer_name = 'Median NDVI',
    
)

p.center_object(pabbi, 10)

p

Map(bottom=6700702.0, center=[34.02510458074524, 71.77439125564787], controls=(WidgetControl(options=['positio…